<a href="https://colab.research.google.com/github/KeshriPooja/PythonCode/blob/master/CovidData_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Covid Data Analysis

In [0]:
import pandas as pd
import numpy as np

In [2]:
import datetime
 
def date_range(start, end):
    r = (end+datetime.timedelta(days=1)-start).days
    print(r)
    return [start+datetime.timedelta(days=i) for i in range(r)]
 
start = datetime.date(2020,1,22)
end = datetime.date(2020,4,23)
dateList = date_range(start, end)
datelist = []
for i in dateList :
  datelist.append(i.strftime("%m-%d-%Y")) 

93


In [0]:
list_data = []
for i in datelist : 
  try :  
    df = pd.read_csv('https://github.com/CSSEGISandData/COVID-19/raw/master/csse_covid_19_data/csse_covid_19_daily_reports/{}.csv'.format(i))
    df['date'] = i
    list_data.append(df)
  except : 
    print(i+'not found')

In [0]:
covid_data= pd.concat(list_data,ignore_index= True)

In [5]:
covid_data.describe()

,Confirmed,Deaths,Recovered,Latitude,Longitude,FIPS,Lat,Long_,Active
count,107960.000000,107538.000000,107591.000000,4799.000000,4799.000000,91193.000000,99068.000000,99068.000000,100362.000000
mean,488.328103,28.958796,122.855704,29.152227,6.297000,30884.144880,36.637395,-81.152157,286.231791
std,5512.393459,527.970306,2241.339138,21.927711,84.758913,16568.296726,10.079272,40.402064,3624.312413
min,0.000000,0.000000,0.000000,-41.454500,-157.858400,66.000000,-51.796300,-170.132000,-80203.000000
25%,1.000000,0.000000,0.000000,21.000000,-75.015200,18111.000000,33.780177,-96.223192,0.000000
50%,7.000000,0.000000,0.000000,35.443700,11.609400,29141.000000,37.890901,-87.839205,0.000000
75%,43.000000,1.000000,0.000000,42.756000,89.046200,46017.000000,41.580717,-81.128570,10.000000
max,213024.000000,25549.000000,103300.000000,72.000000,178.065000,99999.000000,71.706900,178.065000,132223.000000


In [7]:
covid_data.columns

Index(['Province/State', 'Country/Region', 'Last Update', 'Confirmed',
       'Deaths', 'Recovered', 'date', 'Latitude', 'Longitude', 'FIPS',
       'Admin2', 'Province_State', 'Country_Region', 'Last_Update', 'Lat',
       'Long_', 'Active', 'Combined_Key'],
      dtype='object')

## Merge Duplicate Field into one like Province/State and Province_State

In [15]:
covid_data.isnull().sum()

Province/State    103621
Country/Region    100362
Last Update       100362
Confirmed             19
Deaths               441
Recovered            388
date                   0
Latitude          103180
Longitude         103180
FIPS               16786
Admin2             16324
Province_State     13454
Country_Region      7617
Last_Update         7617
Lat                 8911
Long_               8911
Active              7617
Combined_Key        7617
dtype: int64

In [0]:
covid_data['Country_Region']= np.where(covid_data['Country_Region'].isnull(),covid_data['Country/Region'],covid_data['Country_Region'])

In [0]:
covid_data['Province_State']= np.where(covid_data['Province_State'].isnull(),covid_data['Province/State'],covid_data['Province_State'])
covid_data['Latitude']= np.where(covid_data['Latitude'].isnull(),covid_data['Lat'],covid_data['Latitude'])
covid_data['Longitude']= np.where(covid_data['Longitude'].isnull(),covid_data['Long_'],covid_data['Longitude'])

In [20]:
covid_data.isnull().sum()

Province/State    103621
Country/Region    100362
Last Update       100362
Confirmed             19
Deaths               441
Recovered            388
date                   0
Latitude            4112
Longitude           4112
FIPS               16786
Admin2             16324
Province_State      9096
Country_Region         0
Last_Update         7617
Lat                 8911
Long_               8911
Active              7617
Combined_Key        7617
dtype: int64

In [21]:
covid_data.shape

(107979, 18)

In [0]:
covid_data=covid_data.drop(['Province/State','Country/Region','Lat','Long_'],axis='columns')

In [0]:
covid_data[['Confirmed','Deaths','Recovered']]=covid_data[['Confirmed','Deaths','Recovered']].fillna(0)

In [0]:
covid_data['Active']=covid_data['Confirmed']-covid_data['Deaths']-covid_data['Recovered']

In [0]:
covid_data['Death Percentage']=(covid_data['Deaths']*100)/(covid_data['Confirmed']+0.000000001)

In [54]:
covid_data.head(5)

,Last Update,Confirmed,Deaths,Recovered,date,Latitude,Longitude,FIPS,Admin2,Province_State,Country_Region,Last_Update,Active,Combined_Key,Death Percentage
0,1/22/2020 17:00,1.0,0.0,0.0,01-22-2020,NaN,NaN,NaN,NaN,Anhui,Mainland China,NaN,1.0,NaN,0.0
1,1/22/2020 17:00,14.0,0.0,0.0,01-22-2020,NaN,NaN,NaN,NaN,Beijing,Mainland China,NaN,14.0,NaN,0.0
2,1/22/2020 17:00,6.0,0.0,0.0,01-22-2020,NaN,NaN,NaN,NaN,Chongqing,Mainland China,NaN,6.0,NaN,0.0
3,1/22/2020 17:00,1.0,0.0,0.0,01-22-2020,NaN,NaN,NaN,NaN,Fujian,Mainland China,NaN,1.0,NaN,0.0
4,1/22/2020 17:00,0.0,0.0,0.0,01-22-2020,NaN,NaN,NaN,NaN,Gansu,Mainland China,NaN,0.0,NaN,0.0


## Creating a new Column of Firstcase_date when a country_Region has get their first case

In [0]:
Firstcase_date=covid_data.groupby(['Country_Region'])['date'].min()

In [0]:
Firstcase_date=Firstcase_date.to_frame().reset_index().rename(columns={'date':'FirstCase_date'})

In [73]:
Firstcase_date

,Country_Region,FirstCase_date
0,Azerbaijan,02-28-2020
1,Afghanistan,02-24-2020
2,Albania,03-09-2020
3,Algeria,02-25-2020
4,Andorra,03-02-2020
...,...,...
230,Western Sahara,04-05-2020
231,Yemen,04-10-2020
232,Zambia,03-18-2020
233,Zimbabwe,03-20-2020


In [0]:
covid_data=pd.merge(covid_data,Firstcase_date,how='left',left_on='Country_Region',right_on='Country_Region')

In [77]:
covid_data

,Last Update,Confirmed,Deaths,Recovered,date,Latitude,Longitude,FIPS,Admin2,Province_State,Country_Region,Last_Update,Active,Combined_Key,Death Percentage,Firstcase_date,FirstCase_date
0,1/22/2020 17:00,1.0,0.0,0.0,01-22-2020,NaN,NaN,NaN,NaN,Anhui,Mainland China,NaN,1.0,NaN,0.000000,NaN,01-22-2020
1,1/22/2020 17:00,14.0,0.0,0.0,01-22-2020,NaN,NaN,NaN,NaN,Beijing,Mainland China,NaN,14.0,NaN,0.000000,NaN,01-22-2020
2,1/22/2020 17:00,6.0,0.0,0.0,01-22-2020,NaN,NaN,NaN,NaN,Chongqing,Mainland China,NaN,6.0,NaN,0.000000,NaN,01-22-2020
3,1/22/2020 17:00,1.0,0.0,0.0,01-22-2020,NaN,NaN,NaN,NaN,Fujian,Mainland China,NaN,1.0,NaN,0.000000,NaN,01-22-2020
4,1/22/2020 17:00,0.0,0.0,0.0,01-22-2020,NaN,NaN,NaN,NaN,Gansu,Mainland China,NaN,0.0,NaN,0.000000,NaN,01-22-2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107974,NaN,1026.0,6.0,738.0,04-23-2020,-27.4698,153.0251,NaN,NaN,Queensland,Australia,2020-04-24 03:31:35,282.0,"Queensland, Australia",0.584795,NaN,01-23-2020
107975,NaN,438.0,4.0,331.0,04-23-2020,-34.9285,138.6007,NaN,NaN,South Australia,Australia,2020-04-24 03:31:35,103.0,"South Australia, Australia",0.913242,NaN,01-23-2020
107976,NaN,207.0,8.0,67.0,04-23-2020,-42.8821,147.3272,NaN,NaN,Tasmania,Australia,2020-04-24 03:31:35,132.0,"Tasmania, Australia",3.864734,NaN,01-23-2020
107977,NaN,1337.0,16.0,1172.0,04-23-2020,-37.8136,144.9631,NaN,NaN,Victoria,Australia,2020-04-24 03:31:35,149.0,"Victoria, Australia",1.196709,NaN,01-23-2020


In [75]:
covid_data['Total days Since FirstCase']=covid_data['date']

,Last Update,Confirmed,Deaths,Recovered,date,Latitude,Longitude,FIPS,Admin2,Province_State,Country_Region,Last_Update,Active,Combined_Key,Death Percentage,Firstcase_date
0,1/22/2020 17:00,1.0,0.0,0.0,01-22-2020,NaN,NaN,NaN,NaN,Anhui,Mainland China,NaN,1.0,NaN,0.000000,NaN
1,1/22/2020 17:00,14.0,0.0,0.0,01-22-2020,NaN,NaN,NaN,NaN,Beijing,Mainland China,NaN,14.0,NaN,0.000000,NaN
2,1/22/2020 17:00,6.0,0.0,0.0,01-22-2020,NaN,NaN,NaN,NaN,Chongqing,Mainland China,NaN,6.0,NaN,0.000000,NaN
3,1/22/2020 17:00,1.0,0.0,0.0,01-22-2020,NaN,NaN,NaN,NaN,Fujian,Mainland China,NaN,1.0,NaN,0.000000,NaN
4,1/22/2020 17:00,0.0,0.0,0.0,01-22-2020,NaN,NaN,NaN,NaN,Gansu,Mainland China,NaN,0.0,NaN,0.000000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107974,NaN,1026.0,6.0,738.0,04-23-2020,-27.4698,153.0251,NaN,NaN,Queensland,Australia,2020-04-24 03:31:35,282.0,"Queensland, Australia",0.584795,NaN
107975,NaN,438.0,4.0,331.0,04-23-2020,-34.9285,138.6007,NaN,NaN,South Australia,Australia,2020-04-24 03:31:35,103.0,"South Australia, Australia",0.913242,NaN
107976,NaN,207.0,8.0,67.0,04-23-2020,-42.8821,147.3272,NaN,NaN,Tasmania,Australia,2020-04-24 03:31:35,132.0,"Tasmania, Australia",3.864734,NaN
107977,NaN,1337.0,16.0,1172.0,04-23-2020,-37.8136,144.9631,NaN,NaN,Victoria,Australia,2020-04-24 03:31:35,149.0,"Victoria, Australia",1.196709,NaN
